In [ ]:
from model import layers, activations, networks, utils, optimizers, losses, metrics, paddings
import numpy as np
import matplotlib.pyplot as plt
import random

https://neodelphis.github.io/convnet/maths/python/english/2019/07/10/convnet-bp-en.html

In [ ]:
x_train = np.array([
    [
        [[1], [1], [255], [1], [1]],
        [[1], [1], [255], [1], [1]],
        [[1], [1], [255], [1], [1]],
        [[1], [1], [255], [1], [1]],
        [[1], [1], [255], [1], [1]]
    ]
])
x_train = x_train / 255
plt.imshow(x_train[0], cmap='gray')

In [ ]:
y_train = np.array([[0, 0, 1, 0, 0, 0, 0, 0, 0]])
y_train.shape

In [ ]:
model = networks.FeedForward([
    layers.Input(input_shape=(5, 5, 1)),
    layers.Convolution(nr_kernels=1, kernel_size=(3, 3), activation=activations.Sigmoid, padding=paddings.Valid),
    layers.Flatten(),
    layers.Output()
])

In [ ]:
model.compile(
    optimizer=optimizers.sgd(learning_rate=1),
    loss=losses.crossentropy,
    metric=metrics.Accuracy)

In [ ]:
model.summary()

In [ ]:
p = model.predict(x_train[0])
p

In [ ]:
plt.bar(np.linspace(0, 8, 9), p)

In [ ]:
i = model.layers[1].i[:, :, 0]
i

In [ ]:
w = model.layers[1].w[0, 0, :, :]
w

In [ ]:
o = model.layers[1].o[:, :, 0]
o

In [ ]:
loss_untrained = 0.5 * np.sum(np.power(y_train - p, 2))

In [ ]:
L = p - y_train
L

In [ ]:
L_conv = L.reshape(o.shape)
L_conv

In [ ]:
dw = np.zeros(w.shape)

for y in range(3):
    for x in range(3):
        chunk = i[y : y + 3, x : x + 3]
        dw[y, x] = np.sum(chunk * L_conv)

dw

In [ ]:
L_conv_p = paddings.Same(np.expand_dims(L_conv, -1), (5, 5))
L_conv_p = L_conv_p[:, :, 0]
L_conv_p

In [ ]:
dx = np.zeros(i.shape)

for y in range(5):
    for x in range(5):
        dx[y, x] = np.sum(L_conv_p[y : y + 3, x : x + 3] * np.flipud(np.fliplr(w)))

dx

In [ ]:
db = np.sum(L_conv)
db

In [ ]:
model.layers[1].w = model.layers[1].w - dw
model.layers[1].b = model.layers[1].b - db

In [ ]:
y_train

In [ ]:
loss_untrained

In [ ]:
p2 = model.predict(x_train[0])
loss_trained = 0.5 * np.sum(np.power(y_train - p2, 2))
loss_trained

In [ ]:
p2

In [ ]:
plt.bar(np.linspace(0, 8, 9), p2)

In [ ]:
w

In [ ]:
model.layers[1].w[0, 0, :, :]